# Assignment 4

In this assignment, you are required to implement computational node class for building an autograd from scratch.

### General notes:
* Make sure you tagged the cell that contain your solution with "solution" tag
* Make sure this cell result is independent of any other cells. In other words, it will work even if it was the only cell in the notebook.

In [1]:
def solution():
    from uuid import uuid4
    class comp_node:
        def __init__(self, val, children = [], op = "assign"):
            self.val = val
            self.children = children
            self.grad = 0
            self.op = op
            self.backward_prop = lambda : None
            self.identity = uuid4()

        def __to_comp_node(self, obj):
            if not isinstance(obj, comp_node):
                return comp_node(val = obj)
            else:
                return obj
        # you are given an example of doing a subtraction operation
        # and defining a method to do the subtraction backward propagation.
        def __sub__(self, other):
            other = self.__to_comp_node(other)
            out = comp_node(val = self.val - other.val,
                            children = [self, other],
                            op = "sub")
            def _backward_prop():
                self.grad += out.grad * 1
                other.grad += out.grad * (-1)
            out.backward_prop = _backward_prop
            return out
        def __rsub__(self, other):
            other = self.__to_comp_node(other)
            return other - self
        # Implement the add, multiplication, power functions
        def __add__(self, other):
            # write your code here
            other=self.__to_comp_node(other)
            out=comp_node(val=self.val+other.val, children = [self,other],op="add")
            def _backward_prop():
              self.grad +=out.grad*1
              other.grad += out.grad*1

            out.backward_prop=_backward_prop
            # make sure to return the output
            return out
        def __radd__(self, other):
            other = self.__to_comp_node(other)
            return other + self
        def __mul__(self, other):
            # write your code here
            other=self.__to_comp_node(other)
            out=comp_node(val=self.val * other.val, children = [self,other],op="mul")
            def _backward_prop():
              self.grad +=out.grad*other.val
              other.grad += out.grad*self.val
            out.backward_prop=_backward_prop
            # make sure to return the output
            return out
        def __rmul__(self, other):
            other = self.__to_comp_node(other)
            return other * self
        def __pow__(self, exponent):
            if not isinstance(exponent, (int, float)):
                raise ValueError("Unsupported types")
            out = comp_node(val = self.val**(exponent),children=[self],op=f"power{exponent}")
            # write your code here
            def _backward_prop():
              self.grad+=out.grad * (exponent)*self.val**(exponent-1)
            out.backward_prop=_backward_prop

            return out
        def __eq__(self, other):
            return self.val == other.val
        def __repr__(self):
            return f"op: {self.op} | val: {self.val:.4f} | children: {len(self.children)} | grad: {self.grad}"
        def __hash__(self):
            return int(self.identity)
        def topo_sort (self, collect_edges = False):
            res = []
            visited = set()
            if collect_edges : edges = []
            def visit(node):
                if node not in visited:
                    visited.add(node)
                    for child in node.children:
                        if collect_edges:
                            edges.append((child, node))
                        visit(child)
                    res.append(node)
            visit(self)
            if collect_edges:
                return res, edges
            return res
        def backward(self):
            # implement backward function
            # to do back propagation
            # for all nodes
            nodes = self.topo_sort()
            self.grad = 1
            for node in reversed(nodes):
              node.backward_prop()
            return

    # Don't change the return
    return comp_node

Use the cell below to test your solution before submission.

In [3]:
comp_node = solution()
assert comp_node(val = 5).val == 5, "Assignment failed"
assert (comp_node(val = 5) - comp_node(val = 3)).val == 2
assert (comp_node(val = 5) - 3).val == 2
assert (3 - comp_node(val = 5)).val == -2
assert (comp_node(val = 5)**2).val == 25
assert (comp_node(val = 5)**2) == comp_node(val = 25)
assert (comp_node(val = 5) + comp_node(val = 3)).val == 8
assert (comp_node(val = 5) + 3).val == 8
assert (3 + comp_node(val = 5)).val == 8
assert (comp_node(val = 5) * 3).val == 15
assert (3 * comp_node(val = 5)).val == 15
print("All Assignment and operation tests have passed!")

All Assignment and operation tests have passed!
